In [1]:
import pandas as pd
import numpy as np
import re
import time
import os
import shutil

from selenium import webdriver

from tqdm.notebook import tqdm

import json
from datetime import datetime

In [2]:
df = pd.read_json('dados_brutos/parsed_videos_2020_09_25_18_37_50.json', lines=True)
df.head()

,link,title,query
0,https://www.youtube.com/watch?v=J3LHO1EKFuU,"Colloquium Sept 24, 2020 -- Machine Learning f...",machine+learning
1,https://www.youtube.com/watch?v=x6PikWQQep0,Machine Learning in Sinhala | #7 - Reinforceme...,machine+learning
2,https://www.youtube.com/watch?v=9adr8as6qwg,"Learn 𝗔𝗜, 𝗗𝗮𝘁𝗮 𝗦𝗰𝗶𝗲𝗻𝗰𝗲, 𝗠𝗮𝗰𝗵𝗶𝗻𝗲 𝗟𝗲𝗮𝗿𝗻𝗶𝗻𝗴 from ...",machine+learning
3,https://www.youtube.com/watch?v=WEJ65Rvj3lc,Reinforcement learning based dependency resolu...,machine+learning
4,https://www.youtube.com/watch?v=77hMmXZjpC8,"Programming Languages, Software Engineering an...",machine+learning


In [19]:
dummy = df.copy()
dummy['code'] = df['link'].str.extract(r'(v=.*)')
dummy[['link', 'code']]

,link,code
0,https://www.youtube.com/watch?v=J3LHO1EKFuU,v=J3LHO1EKFuU
1,https://www.youtube.com/watch?v=x6PikWQQep0,v=x6PikWQQep0
2,https://www.youtube.com/watch?v=9adr8as6qwg,v=9adr8as6qwg
3,https://www.youtube.com/watch?v=WEJ65Rvj3lc,v=WEJ65Rvj3lc
4,https://www.youtube.com/watch?v=77hMmXZjpC8,v=77hMmXZjpC8
...,...,...
634,https://www.youtube.com/watch?v=em8nBc-zRaM,v=em8nBc-zRaM
635,https://www.youtube.com/watch?v=unc2SqE_jXQ,v=unc2SqE_jXQ
636,https://www.youtube.com/watch?v=aJC7YeDFxZw,v=aJC7YeDFxZw
637,https://www.youtube.com/watch?v=rAFPtCOUfVc,v=rAFPtCOUfVc
